In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 80.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 88.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
import json

with open('/kaggle/input/rubq-20/RuBQ_2.0_paragraphs.json', 'r') as f:
    data = json.load(f)

In [3]:
# requirements.txt:
# chromadb sentence-transformers torch

import chromadb
from chromadb.utils import embedding_functions

In [ ]:
ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sergeyzh/rubert-mini-frida", device="cuda"
)

In [ ]:
chroma_client.delete_collection('docs')

In [4]:
chroma_client = chromadb.Client()
collection = chroma_client.create_collection(name="docs",
                                            # embedding_function=ef
                                            )

# 3. Добавление документов
documents = [
    "Наша компания основана в 2020 году",
    "Главный офис находится в Москве", 
    "Мы разрабатываем ИИ-решения для бизнеса",
    "Стоимость подписки - 1000 рублей в месяц"
]

In [5]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel


def pool(hidden_state, mask, pooling_method="mean"):
    if pooling_method == "mean":
        s = torch.sum(hidden_state * mask.unsqueeze(-1).float(), dim=1)
        d = mask.sum(axis=1, keepdim=True).float()
        return s / d
    elif pooling_method == "cls":
        return hidden_state[:, 0]

inputs = documents

tokenizer = AutoTokenizer.from_pretrained("sergeyzh/rubert-mini-frida")
model = AutoModel.from_pretrained("sergeyzh/rubert-mini-frida")

tokenized_inputs = tokenizer(inputs, max_length=512, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**tokenized_inputs)
    
embeddings = pool(
    outputs.last_hidden_state, 
    tokenized_inputs["attention_mask"],
    pooling_method="mean"
)

# embeddings = F.normalize(embeddings, p=2, dim=1)
# sim_scores = embeddings[:3] @ embeddings[3:].T
# print(sim_scores.diag().tolist())
# [0.9423348903656006, 0.8306248188018799, 0.7095720767974854]
# [0.9360030293464661, 0.8591322302818298, 0.728583037853241] - FRIDA


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

...

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

In [7]:
collection.add(
    embeddings=embeddings.numpy(),
    documents=documents,
    ids=[f"doc_{i}" for i in range(len(documents))]
)

In [8]:
collection.get()

{'ids': ['doc_0', 'doc_1', 'doc_2', 'doc_3'],
 'embeddings': None,
 'documents': ['Наша компания основана в 2020 году',
  'Главный офис находится в Москве',
  'Мы разрабатываем ИИ-решения для бизнеса',
  'Стоимость подписки - 1000 рублей в месяц'],
 'uris': None,
 'included': ['metadatas', 'documents'],
 'data': None,
 'metadatas': [None, None, None, None]}

In [11]:
inputs = ["где находится офис?",]
tokenized_inputs = tokenizer(inputs, max_length=512, padding=True, truncation=True, return_tensors="pt")

with torch.no_grad():
    outputs = model(**tokenized_inputs)
    
embeddings = pool(
    outputs.last_hidden_state, 
    tokenized_inputs["attention_mask"],
    pooling_method="mean"
)

In [15]:
collection.query(
    query_embeddings=embeddings.numpy(),
    n_results=1
)

{'ids': [['doc_1']],
 'embeddings': None,
 'documents': [['Главный офис находится в Москве']],
 'uris': None,
 'included': ['metadatas', 'documents', 'distances'],
 'data': None,
 'metadatas': [[None]],
 'distances': [[8.31290054321289]]}

In [ ]:
# 4. Поиск
def search_docs(question):
    results = collection.query(
        query_texts=[question],
        n_results=5
    )
    # print(results)
    return results['documents'][0]

# Использование
results = search_docs("Сколько раз Челси выигрывал Кубок Англии?")
print("Найденные документы:", *results, sep='\n----\n')